In [103]:
import re
import nltk
import heapq  
import pandas as pd
import csv
import numpy as np
from bs4 import BeautifulSoup


In [104]:
def buildCSV():
    website_url = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population').text
    soup = BeautifulSoup(website_url,'lxml')
    mytable = soup.find('table',{'class':'wikitable sortable'})
    rows = mytable.findAll('tr')
    import re
    links = []
    for row in rows[1:]:
        links.append('https://en.wikipedia.org' + str(row.find('a',attrs={'href': re.compile("wiki/")}).get('href')))
    headers = rows[0].findAll('th')
    f = open('abc.csv',"w",encoding='utf-8-sig')
    writeheaders(headers,f)
    writeData(rows,f)
    f.close()
    return links

In [105]:
def writeheaders(headers,f):
    heads = []
    for h in headers:
        txt = str(h.get_text().strip().split('[')[0])
        txt = txt.title().replace(' ','')
        if txt[:4] == '2016':
            txt1 = txt + 'Mi'
            txt2 = txt + 'Km'
            heads.append(txt1)
            heads.append(txt2)
            continue
        heads.append(txt)
    heads[-1] = 'latitude'
    heads.append('longitude')
    headerstr = ','.join(heads) + '\n'
    f.write(headerstr)

In [106]:
def writeData(rows,f):
    from unicodedata import normalize
    for r in rows[1:]:
        txt = str(r.text)
        txtlist = str(txt.replace('\n\n','\n')).strip().split('\n')
        for i in range(len(txtlist)):
            txtlist[i] = txtlist[i].replace(',','')
            txtlist[i] =normalize('NFKD', txtlist[i])
            txtlist[i] = txtlist[i].strip()
            txtlist[i] = txtlist[i].split('[')[0]
        lat = (txtlist[-1].split()[0]).replace("′","")
        lat = lat.replace("°","")[:-1]
        long = (txtlist[-1].split()[1]).replace("′","")
        long = long.replace("°","")[:-2]
        txtlist[-1] = lat
        txtlist.append(long)
        writestr = (','.join(txtlist)) + '\n'
        f.write(writestr)

In [112]:
def addFIPSandSummary(data, links):
    fips = []
    summ = []
    for l in links:
        summ.append(generateSummary(l))
        ctr += 1
        flag = 0
        link = requests.get(l).text
        soup = BeautifulSoup(link,'lxml')
        tb = soup.find('table',{'class':'infobox geography vcard'})
        rows = tb.findAll('tr')
        for row in rows:
            if str(row.text)[:4] == 'FIPS':
                td = row.find('td')
                text = str(td.text).replace('-','').split('[')[0][:7]
                fips.append('https://factfinder.census.gov/bkmk/table/1.0/en/DEC/10_DP/DPDP1/1600000US'+text)
                flag = 1
        if flag == 0:
            fips.append(np.nan)
    data['FIPS Code'] = fips
    data['summary'] = summ
    return data

In [118]:
def generateSummary(l):
    link = requests.get(l).text
    soup = BeautifulSoup(link,'lxml')
    pg = soup.findAll('p')
    text = ""
    for p in pg:
        text += str(p.text)
    text = re.sub(r'\[[0-9]*\]', ' ', text)  
    text = re.sub(r'\s+', ' ', text)  
    ftext = re.sub('[^a-zA-Z]', ' ', text )  
    ftext = re.sub(r'\s+', ' ', ftext)  

    sentence_list = nltk.sent_tokenize(text)  

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}  
    for word in nltk.word_tokenize(ftext):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())
    word_frequencies.values()
    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)
    summary = ''.join(summary_sentences)
    summary.replace(',','')
    return summary
    

In [115]:
def main():
    links = buildCSV()
    data = pd.read_csv('abc.csv', encoding = 'utf-8')
    data = data.set_index('2018Rank')
    data = addFIPSandSummary(data, links)
    data.to_csv('Final.csv')

In [117]:
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27